In [1]:
import planetary_computer
import itertools
import dask.dataframe as dd

cc = planetary_computer.get_container_client("pcstacitems", "items")

blobs = list(cc.list_blobs("sentinel-2-l2a.parquet/"))

def key(blob):
    return blob.name.split("/")[1].split("_")[0]

keep_blobs = []
for k, v in itertools.groupby(sorted(blobs, key=key), key=key):
    v = list(v)
    blob = max(v, key=lambda x: x.last_modified)
    keep_blobs.append(blob)
    
uris = [f"az://items/{blob.name}" for blob in keep_blobs]

In [2]:
len(uris)

445

In [11]:
df = dd.read_parquet(uris, storage_options={"account_name": "pcstacitems", "credential": planetary_computer.sas.get_token("pcstacitems", "items").token})
df.head()

,type,stac_version,stac_extensions,id,geometry,bbox,links,assets,collection,datetime,...,s2:cloud_shadow_percentage,s2:nodata_pixel_percentage,s2:unclassified_percentage,s2:dark_features_percentage,s2:not_vegetated_percentage,s2:degraded_msi_data_percentage,s2:high_proba_clouds_percentage,s2:reflectance_conversion_factor,s2:medium_proba_clouds_percentage,s2:saturated_defective_pixel_percentage
0,Feature,1.0.0,[https://stac-extensions.github.io/eo/v1.0.0/s...,S2A_MSIL2A_20150704T101006_R022_T35XQA_2021041...,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x1e\x00...","[32.782321671748434, 71.80556197526644, 36.189...",[{'href': 'https://planetarycomputer.microsoft...,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...",sentinel-2-l2a,2015-07-04 10:10:06.027000+00:00,...,0.000000,20.722227,0.000000,0.000000,0.000000,0.0,92.546540,0.967449,4.807670,0.0
1,Feature,1.0.0,[https://stac-extensions.github.io/eo/v1.0.0/s...,S2A_MSIL2A_20150704T101006_R022_T32TMM_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0e\x00...,"[8.479220766509275, 41.46069640920803, 9.11869...",[{'href': 'https://planetarycomputer.microsoft...,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...",sentinel-2-l2a,2015-07-04 10:10:06.027000+00:00,...,0.009120,63.654864,0.265107,0.215319,13.662070,0.0,0.048035,0.967449,0.051376,0.0
2,Feature,1.0.0,[https://stac-extensions.github.io/eo/v1.0.0/s...,S2A_MSIL2A_20150704T101006_R022_T32TMN_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0c\x00...,"[8.765144061005621, 42.36232117367103, 9.12043...",[{'href': 'https://planetarycomputer.microsoft...,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...",sentinel-2-l2a,2015-07-04 10:10:06.027000+00:00,...,0.001858,85.355914,0.437180,0.223688,19.642879,0.0,0.011238,0.967449,0.022928,0.0
3,Feature,1.0.0,[https://stac-extensions.github.io/eo/v1.0.0/s...,S2A_MSIL2A_20150704T101006_R022_T36WWC_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\t\x00\x...,"[32.99946824095416, 69.68910488955622, 34.0680...",[{'href': 'https://planetarycomputer.microsoft...,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...",sentinel-2-l2a,2015-07-04 10:10:06.027000+00:00,...,0.000432,88.475966,0.323290,0.006248,0.008810,0.0,65.812266,0.967449,19.050561,0.0
4,Feature,1.0.0,[https://stac-extensions.github.io/eo/v1.0.0/s...,S2A_MSIL2A_20150704T101006_R022_T36WWD_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\r\x00\x...,"[32.999443871575316, 70.21094250231056, 35.756...",[{'href': 'https://planetarycomputer.microsoft...,"{'AOT': {'gsd': 10.0, 'href': 'https://sentine...",sentinel-2-l2a,2015-07-04 10:10:06.027000+00:00,...,0.000000,39.018765,0.000000,0.000000,0.001485,0.0,97.629422,0.967449,1.861097,0.0


In [12]:
df = df[['id', 'geometry', 'datetime', 'eo:cloud_cover']]

In [13]:
df.head()

,id,geometry,datetime,eo:cloud_cover
0,S2A_MSIL2A_20150704T101006_R022_T35XQA_2021041...,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x1e\x00...",2015-07-04 10:10:06.027000+00:00,97.851394
1,S2A_MSIL2A_20150704T101006_R022_T32TMM_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0e\x00...,2015-07-04 10:10:06.027000+00:00,0.177088
2,S2A_MSIL2A_20150704T101006_R022_T32TMN_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0c\x00...,2015-07-04 10:10:06.027000+00:00,0.034959
3,S2A_MSIL2A_20150704T101006_R022_T36WWC_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\t\x00\x...,2015-07-04 10:10:06.027000+00:00,88.028410
4,S2A_MSIL2A_20150704T101006_R022_T36WWD_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\r\x00\x...,2015-07-04 10:10:06.027000+00:00,99.703225


In [14]:
mask = (df["eo:cloud_cover"] < 20)
df = df[mask]

In [15]:
df.head()

,id,geometry,datetime,eo:cloud_cover
1,S2A_MSIL2A_20150704T101006_R022_T32TMM_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0e\x00...,2015-07-04 10:10:06.027000+00:00,0.177088
2,S2A_MSIL2A_20150704T101006_R022_T32TMN_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0c\x00...,2015-07-04 10:10:06.027000+00:00,0.034959
7,S2A_MSIL2A_20150704T101006_R022_T31RGL_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,2015-07-04 10:10:06.027000+00:00,1.548134
8,S2A_MSIL2A_20150704T101006_R022_T31RGM_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,2015-07-04 10:10:06.027000+00:00,0.002084
9,S2A_MSIL2A_20150704T101006_R022_T31RGN_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0c\x00...,2015-07-04 10:10:06.027000+00:00,0.000000


In [16]:
df["datetime"] = dd.to_datetime(df["datetime"])

In [17]:
df.head()

,id,geometry,datetime,eo:cloud_cover
1,S2A_MSIL2A_20150704T101006_R022_T32TMM_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0e\x00...,2015-07-04 10:10:06.027000+00:00,0.177088
2,S2A_MSIL2A_20150704T101006_R022_T32TMN_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0c\x00...,2015-07-04 10:10:06.027000+00:00,0.034959
7,S2A_MSIL2A_20150704T101006_R022_T31RGL_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,2015-07-04 10:10:06.027000+00:00,1.548134
8,S2A_MSIL2A_20150704T101006_R022_T31RGM_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,2015-07-04 10:10:06.027000+00:00,0.002084
9,S2A_MSIL2A_20150704T101006_R022_T31RGN_2021041...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0c\x00...,2015-07-04 10:10:06.027000+00:00,0.000000


In [19]:
filtered_df = df[df['datetime'] > '2021-01-01']

In [20]:
filtered_df = filtered_df.compute()

In [22]:
filtered_df.shape

(3859047, 4)

In [23]:
filtered_df.head()

,id,geometry,datetime,eo:cloud_cover
26779,S2A_MSIL2A_20210104T094411_R036_T32RQN_2021011...,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0b\x00...",2021-01-04 09:44:11.024000+00:00,2.850817
26780,S2A_MSIL2A_20210104T094411_R036_T32RQP_2021010...,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0b\x00...",2021-01-04 09:44:11.024000+00:00,0.248822
26785,S2A_MSIL2A_20210104T094411_R036_T33SXS_2021010...,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0b\x00...",2021-01-04 09:44:11.024000+00:00,7.399789
26786,S2A_MSIL2A_20210104T094411_R036_T33SXT_2021011...,"b""\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0b\x00...",2021-01-04 09:44:11.024000+00:00,3.925122
26787,S2A_MSIL2A_20210104T094411_R036_T33SXU_2021010...,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x07\x00...,2021-01-04 09:44:11.024000+00:00,1.881726


In [ ]:
filtered_df.to_parquet("s2l2a_2_5_2024_clouds_lt_20_date_gt_2021_01_01.parquet", index=False)